In [1]:
#M4.4XLarge EC2
import pandas as pd
import numpy as np
import re
import string
import sys
import os

import matplotlib.pyplot as plt

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

from collections import Counter

%matplotlib inline

In [2]:
projects = pd.read_csv('./data/opendata_projects.csv') #878852 Entries

In [3]:
print "Benchmarks::"
projects.funding_status.value_counts()/projects.shape[0]

Benchmarks::


completed      0.674467
expired        0.288339
live           0.027944
reallocated    0.009250
Name: funding_status, dtype: float64

In [4]:
projects = projects.loc[(projects.funding_status == 'completed') | (projects.funding_status == 'expired')] #846164

In [6]:
#Should generate time stamp as real date for visuals

projects = projects[project_cols_subset]
projects.set_index('_projectid', inplace =True)
projects['Year'] = projects.date_posted.map(lambda d: d[:4]).astype('int') #Should be String but converting as Int for now
projects['Month'] = projects.date_posted.map(lambda d: d[5:7]).astype('int') #Should be String but converting as Int for now

In [ ]:
#Fix Date Times
for c in projects.columns:
    if 'date' in c:
        projects[c] = pd.to_datetime(projects[c], format="%Y-%m-%d %H:%M:%S")

In [5]:
# project_cols_subset = ['_projectid', '_teacher_acctid', '_schoolid', 
#                        'school_latitude', 'school_longitude', 
#                        'school_city', 'school_state', 'school_zip', 'school_metro',
#                        'school_charter', 'school_magnet', 'school_year_round',
#                        'school_nlns', 'school_kipp', 'school_charter_ready_promise',
#                        'teacher_prefix', 'teacher_teach_for_america', 'teacher_ny_teaching_fellow',
#                        'primary_focus_subject', 'primary_focus_area',
#                        'secondary_focus_subject', 'secondary_focus_area',
#                        'resource_type', 'poverty_level', 'grade_level', 
#                        'total_price_excluding_optional_support', 'total_price_including_optional_support',
#                        'students_reached', 'total_donations', 'num_donors',
#                        'eligible_double_your_impact_match', 'eligible_almost_home_match', 'funding_status',
#                        'date_posted', 'date_completed', 'date_thank_you_packet_mailed'] #36 Columns

In [7]:
projects.students_reached.fillna(value = projects.students_reached.median(), inplace = True)

In [8]:
model_cols_subset = ['school_state', 'school_metro',
                       'school_charter', 'school_magnet', 'school_year_round',
                       'school_nlns', 'school_kipp', 'school_charter_ready_promise',
                       'teacher_prefix', 'teacher_teach_for_america', 'teacher_ny_teaching_fellow',
                       'primary_focus_subject', 'primary_focus_area',
                       'secondary_focus_subject', 'secondary_focus_area',
                       'resource_type', 'poverty_level', 'grade_level', 
                       'total_price_excluding_optional_support', 'total_price_including_optional_support',
                       'students_reached','Year', 'Month']
model_target = ['funding_status']

In [9]:
model_df = projects[model_cols_subset + model_target] #Early phase model

enc = {}
for c in model_df.columns:
    if model_df[c].dtype == 'object':
        enc[c] = LabelEncoder()
        model_df[c] = enc[c].fit_transform(model_df[c])

/home/ubuntu/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [10]:
# X = model_df[model_cols_subset].values
# y = model_df[model_target].values.ravel()

# indices = np.arange(X.shape[0])
# np.random.shuffle(indices)

# X = X[indices]
# y = y[indices]

# X_train, X_test, y_train, y_test = train_test_split(X, y)

# rf = RandomForestClassifier(n_estimators=50, oob_score=True)
# rf.fit(X_train, y_train)

# for f in sorted(zip(model_cols_subset, rf.feature_importances_), key = lambda x: x[1], reverse = True):
#     print f

#print rf.oob_score_

In [11]:
#Looking at resources requested for each project

f = open('./data/opendata_resources.csv')
header = f.readline().split(',')

header = [c.strip() for c in header]
resources_raw = f.read()
f.close()

resources_row_text = re.split('"\n"', resources_raw)
resouces_for_df = []
for row in resources_row_text:
    resouces_for_df.append(re.split('","', row))
resources = pd.DataFrame(resouces_for_df, columns=header)
del resources_raw
del resources_row_text
del resouces_for_df

In [12]:
resources.iloc[0,0] = resources.iloc[0,0][1:] #fixes the parsing issue
resources.iloc[-1,-1] = 1 #fixes the parsing issue

In [13]:
resources.set_index('_projectid', inplace = True)

resources['vendorid'] = resources.vendorid.map(lambda x: -1 if x == '' else x).astype('int')
resources['vendor_name'] = resources.vendor_name.map(lambda x: 'Unknown' if x == '' else x)
resources['item_name'] = resources.item_name.map(lambda x: 'Unknown' if x == '' else x)
resources['item_number'] = resources.item_number.map(lambda x: -1 if x == '' else x)
resources['item_unit_price'] = resources.item_unit_price.map(lambda x: 14.00 if x == '' else x).astype('float') #median = 13.86 
resources['item_quantity'] = resources.item_quantity.map(lambda x: 1 if x == '' else x).astype('int') #median = 1

resources['price times quantity'] = resources.item_unit_price * resources.item_quantity


In [14]:
#Helper Functions
def join_text(list_str):
    return ' '.join(list_str)

def only_words(s):
    s = s.lower().translate(None, string.punctuation + string.digits)
    return ''.join([c for c in s if c in string.ascii_lowercase + ' '])

In [15]:
resources_gb = resources.groupby(resources.index)
resources_agg = resources_gb.agg({'item_unit_price': [np.min, np.max], 'item_quantity':[np.sum], 'item_name':[join_text], 'price times quantity':[np.sum] })
resources_agg[('item_name','join_text')] = resources_agg[('item_name','join_text')].map(only_words)

resources_agg['avg_price'] = resources_agg[('price times quantity', 'sum')]/resources_agg[('item_quantity','sum')]
#resources_agg.head()

In [16]:
print resources_agg.shape
print model_df.shape

(878528, 6)
(846164, 24)


In [17]:
model_df = model_df.join(resources_agg, how = 'inner')

In [18]:
del projects
del resources_agg

In [19]:
X = model_df.drop([('item_name', 'join_text')] + model_target, axis = 1).values
y = model_df[model_target].values.ravel()


In [20]:
model_df[('item_name', 'join_text')].head()


_projectid
0000023f507999464aa2b78875b7e5d6    lead  mm  pentel super highpolymer leads mediu...
00000ce845c00cbf0686c992fc369df4                         sony mdr zx blk   headphones
00005f52c98eeaf92b2414a352b023a4    viewsonic pjd  dlp projector  d avervision u  ...
0000702ee4cefdb1e7f89084e50d3c85    xrs dlp proj svga   sharp  xrs multimedia dlp ...
00008c61582d7b4d3a2d89ad88f9eedf     famous poems roy j cook art over  works from ...
Name: (item_name, join_text), dtype: object

In [21]:
if os.path.exists('./model/resource_count_vect.pkl'):
    count_vect = joblib.load('./model/resource_count_vect.pkl')
else:
    print 'Error No Vectorizer'
    #Taken From Resources
    #resource_count_vect = CountVectorizer(stop_words='english', max_features=10000, token_pattern=u'(?u)\\b\\w\\w{2,}\\b') #3 or more letters
    #X = resource_count_vect.fit_transform(data[('item_name', 'join_text')])
    #joblib.dump(resource_count_vect, './model/resource_count_vect.pkl') 
if os.path.exists('./model/resource_nmf_5000.pkl'):
    nmf = joblib.load('./model/resource_nmf_5000.pkl')
else:
    print 'Error No NMF'
#     resource_nmf_5000 = NMF(n_components = 50, max_iter= 10, random_state=42) #Should be 10000
#     X_nmf = resource_nmf_5000.fit_transform(X)
#     joblib.dump(resource_nmf_5000, './model/resource_nmf_5000.pkl') 

In [22]:
X_text = count_vect.transform(model_df[('item_name', 'join_text')])
X_text = nmf.transform(X_text)

In [23]:
X = np.hstack((X, X_text))

In [ ]:
indices = np.arange(X.shape[0])

np.random.shuffle(indices,)

X = X[indices]
y = y[indices]

X_train, X_test, y_train, y_test = train_test_split(X, y)

rf = RandomForestClassifier(n_estimators=25, oob_score=True, n_jobs = -1)
rf.fit(X_train, y_train)

print rf.score(X_test, y_test)
print rf.oob_score_

In [24]:
for f in sorted(zip(model_df.columns + , rf.feature_importances_), key = lambda x: x[1], reverse = True):
    print f

0.716504303414
0.701422966397


/home/ubuntu/anaconda/lib/python2.7/site-packages/sklearn/ensemble/forest.py:403: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


In [25]:
print rf.score(X_test, y_test)
print rf.oob_score_

0.716504303414
0.701422966397
